# Homework Starter — Stage 05: Data Storage
Name: Jay Yang

Date: 8.19

Objectives:
- Env-driven paths to `data/raw/` and `data/processed/`
- Save CSV and Parquet; reload and validate
- Abstract IO with utility functions; document choices

In [14]:
import os, pathlib, datetime as dt
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
RAW = pathlib.Path(os.getenv('DATA_DIR_RAW', 'data/raw'))
PROC = pathlib.Path(os.getenv('DATA_DIR_PROCESSED', 'data/processed'))
RAW.mkdir(parents=True, exist_ok=True)
PROC.mkdir(parents=True, exist_ok=True)
print('RAW ->', RAW.resolve())
print('PROC ->', PROC.resolve())

RAW -> /Users/jay/bootcamp_Jingdong_Yang/homework/stage05_data-storage/data/raw
PROC -> /Users/jay/bootcamp_Jingdong_Yang/homework/stage05_data-storage/data/processed


## 1) Create or Load a Sample DataFrame
You may reuse data from prior stages or create a small synthetic dataset.

In [15]:
import numpy as np
dates = pd.date_range('2024-01-01', periods=20, freq='D')
df = pd.DataFrame({'date': dates, 'ticker': ['AAPL']*20, 'price': 150 + np.random.randn(20).cumsum()})
df.head()

,date,ticker,price
0,2024-01-01,AAPL,150.341507
1,2024-01-02,AAPL,148.775331
2,2024-01-03,AAPL,147.187894
3,2024-01-04,AAPL,147.769086
4,2024-01-05,AAPL,148.318705


## 2) Save CSV to data/raw/ and Parquet to data/processed/ (TODO)
- Use timestamped filenames.
- Handle missing Parquet engine gracefully.

In [16]:
def get_timestamp():
    return dt.datetime.now().strftime('%Y%m%d_%H%M%S')

def timestamped_filename(base: str, ext: str) -> str:
    return f"{base}_{get_timestamp()}.{ext}"

BASE_NAME = "stock_data"

# CSV -> data/raw/
csv_path = RAW / timestamped_filename(BASE_NAME, "csv")
df.to_csv(csv_path, index=False)
print(f"\nSaved CSV to: {csv_path}")

# Parquet -> data/processed/
pq_path = PROC / timestamped_filename(BASE_NAME, "parquet")
try:
    df.to_parquet(pq_path, engine='pyarrow')
    print(f"Saved Parquet to: {pq_path}")
except Exception as e:
    print(f"\nParquet save failed (install pyarrow): {e}")
    pq_path = None

pq_path


Saved CSV to: data/raw/stock_data_20250820_182631.csv
Saved Parquet to: data/processed/stock_data_20250820_182631.parquet


PosixPath('data/processed/stock_data_20250820_182631.parquet')

## 3) Reload and Validate (TODO)
- Compare shapes and key dtypes.

In [17]:
def validate_loaded(original, reloaded):
    checks = {
        'shape_equal': original.shape == reloaded.shape,
        'date_is_datetime': pd.api.types.is_datetime64_any_dtype(reloaded['date']) if 'date' in reloaded.columns else False,
        'price_is_numeric': pd.api.types.is_numeric_dtype(reloaded['price']) if 'price' in reloaded.columns else False,
    }
    return checks

df_csv = pd.read_csv(csv_path, parse_dates=['date'])
validate_loaded(df, df_csv)

{'shape_equal': True, 'date_is_datetime': True, 'price_is_numeric': True}

In [18]:
if pq_path:
    try:
        df_pq = pd.read_parquet(pq_path)
        print(validate_loaded(df, df_pq))
    except Exception as e:
        print('Parquet read failed:', e)

{'shape_equal': True, 'date_is_datetime': True, 'price_is_numeric': True}


## 4) Utilities (TODO)
- Implement `detect_format`, `write_df`, `read_df`.
- Use suffix to route; create parent dirs if needed; friendly errors for Parquet.

In [19]:
import typing as t, pathlib

def detect_format(path: t.Union[str, pathlib.Path]):
    s = str(path).lower()
    if s.endswith('.csv'): return 'csv'
    if s.endswith('.parquet') or s.endswith('.pq') or s.endswith('.parq'): return 'parquet'
    raise ValueError('Unsupported format: ' + s)

def write_df(df: pd.DataFrame, path: t.Union[str, pathlib.Path]):
    p = pathlib.Path(path); p.parent.mkdir(parents=True, exist_ok=True)
    fmt = detect_format(p)
    if fmt == 'csv':
        df.to_csv(p, index=False)
    else:
        try:
            df.to_parquet(p)
        except Exception as e:
            raise RuntimeError('Parquet engine not available. Install pyarrow or fastparquet.') from e
    return p

def read_df(path: t.Union[str, pathlib.Path]):
    p = pathlib.Path(path)
    fmt = detect_format(p)
    if fmt == 'csv':
        return pd.read_csv(p, parse_dates=['date']) if 'date' in pd.read_csv(p, nrows=0).columns else pd.read_csv(p)
    else:
        try:
            return pd.read_parquet(p)
        except Exception as e:
            raise RuntimeError('Parquet engine not available. Install pyarrow or fastparquet.') from e

# Demo
p_csv = RAW / f"util_{get_timestamp()}.csv"
p_pq  = PROC / f"util_{get_timestamp()}.parquet"
write_df(df, p_csv)
print(f"CSV utility test passed: {df.equals(read_df(p_csv))}")

try:
    write_df(df, p_pq)
    print(f"Parquet utility test passed: {df.equals(read_df(p_pq))}")
except RuntimeError as e:
    print('Skipping Parquet util demo:', e)

CSV utility test passed: True
Parquet utility test passed: True


## 5) Documentation (TODO)
- Update README with a **Data Storage** section (folders, formats, env usage).
- Summarize validation checks and any assumptions.